In [26]:
from sklearn import datasets
import pandas as pd
import numpy as np
from collections import Counter
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [27]:
iris = datasets.load_iris()
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state = 1)

# Decision Tree Node

In [28]:
class Decision_tree_node:
    def __init__(self,level,y,gain_ratio=None,point=None,feature=None):
        self.level = level
        self.y = y
        self.gain_ratio = gain_ratio
        self.feature = feature
        self.point = point
        self.right = None
        self.left = None

# Decision Tree Construction

### ==>This function calculates the entropy

In [29]:
def entropy_calculator(y):
    dic = Counter(y)
    entropy = 0
    keys = dic.keys()
    total = len(y)
    for i in keys:
        key_value = dic[i]
        pi = (key_value/total)
        entropy+=(-1)*(pi*math.log(pi))/(math.log(2))
    return entropy

### ==>This function calculates the split number for each point

In [30]:
def split_number(d,d1,d2):
    num1 = d1/d
    num2 = d2/d
    return (-1)*((num1*math.log(num1)/math.log(2))+(num2*math.log(num2)/math.log(2)))

### ==>This function returns the information gain of each node

In [31]:
def information_gain(entropy_old,entropy_new):
    return entropy_old-entropy_new

### ==>This function calculates the maximum gain ratio for each point in a particular feature

In [32]:
def gain_on_spliting_point(Xi,y,point,entropy_old):
    temp = y.copy()
    less_than_spliting_point = []
    greater_or_equal_than_spliting_point = []
    
    for i in range(len(Xi)):
        if(Xi[i]<point):
            less_than_spliting_point.append(y[i])
        else:
            greater_or_equal_than_spliting_point.append(y[i])
            
    less_than_spliting_point = np.array(less_than_spliting_point)
    greater_or_equal_than_spliting_point = np.array(greater_or_equal_than_spliting_point)
    
    entropy_less_than = entropy_calculator(less_than_spliting_point)
    entropy_greater_than = entropy_calculator(greater_or_equal_than_spliting_point)
    
    d = len(y)
    d1 = len(less_than_spliting_point)
    d2 = len(greater_or_equal_than_spliting_point)
    
    entropy_new = (d1/d)*entropy_less_than+(d2/d)*entropy_greater_than
    info_gain = information_gain(entropy_old,entropy_new)
    split_num = split_number(d,d1,d2)
    
    gain_ratio = info_gain/split_num
    return gain_ratio

### ==>This function retruns the best splitting point in particular feature

In [33]:
def spliting_point(Xi,y,entropy_old):
    max_GR = 0
    best_spliting_point = None
    temp = Xi.copy()
    temp.sort()
    for i in range(Xi.shape[0]-1):
        point = (temp[i]+temp[i+1])/2
        if(point==temp[i]):
            continue
        Max = gain_on_spliting_point(Xi,y,point,entropy_old)
        if(max_GR<Max):
            max_GR = Max
            best_spliting_point = point
    return max_GR,best_spliting_point

### ==>This function returns the best feature along with best splitting point for a particular node

In [34]:
def gain_ratio(x,y):
    entropy_old = entropy_calculator(y)
    max_gain_ratio = 0
    best_feature = None
    best_spliting_point = None
    for i in range(x.shape[1]):
        gain_ratio,point = spliting_point(x[:,i],y,entropy_old)
        if (gain_ratio>max_gain_ratio):
            max_gain_ratio = gain_ratio
            best_feature = i
            best_spliting_point = point
    return max_gain_ratio,best_spliting_point,best_feature

### ==>This function CREATES THE DECISION TREE

In [35]:
def construct_tree(x,y,level = 0):
    if(len(Counter(y))==1):
        return Decision_tree_node(level,y,None,None,None)
    best_gain_ratio,spliting_point,feature = gain_ratio(x,y)
    node = Decision_tree_node(level,y,best_gain_ratio,spliting_point,feature)
    
    right_node = []
    left_node = []
    y_left = []
    y_right = []
    for i  in range(x.shape[0]):
        if (x[i,feature]<spliting_point):
            left_node.append(x[i])
            y_left.append(y[i])
        else:
            right_node.append(x[i])
            y_right.append(y[i])
    right_node = np.array(right_node)
    left_node = np.array(left_node)
    y_left = np.array(y_left)
    y_right = np.array(y_right)
    
    node.left = construct_tree(left_node,y_left,level+1)
    node.right = construct_tree(right_node,y_right,level+1)
    
    return node

# Printing Function Of Decision Tree 

In [36]:
def count(y):
    dic = Counter(y)
    key = dic.keys()
    d = []
    for i in range(len(dic)):
        d.append(dic[i])
    d =  np.array(d)
    return d

In [37]:
def print_tree(node,feature_names):
    entropy = entropy_calculator(node.y)
    
    if(len(Counter(node.y))==1):
        print("Level",node.level)
        print("Count of",node.y[0],"=",len(node.y))
        print("Current Entropy is =",entropy)
        print("Reached leaf Node \n")
        return
    
    d = Counter(node.y)
    key = set(node.y)
    print("Level",node.level)
    for i in key:
        print("count of",i,"=",d[i])
    print("Current Entropy is =",entropy)
    print("Splitting on feature",feature_names[node.feature],"with gain ratio",node.gain_ratio,"\n")
    
    print_tree(node.left,feature_names)
    print_tree(node.right,feature_names)
    
    

# Predict Function Of Decision Tree

In [38]:
def predict_for_one_input(x,node):
    
    while(node.point!=None):
        point = node.point
        feature = node.feature
        if(x[feature]<point):
            node = node.left
        else:
            node = node.right
    return max(node.y,key = Counter(node.y).get)       

In [39]:
def predict(x,node):
    y = []
    for i in range(x.shape[0]):
        y.append(predict_for_one_input(x[i],node))
    y = np.array(y)
    return y

###                           ==========================DECISION TREE IMPLEMENTATION OVER==========================




# Testing on dummy data (OR GATE)

In [40]:
x = np.array([[1,1],[0,1],[1,0],[0,0]])
y = np.array([1,1,1,0])
feature_name = np.array(["x1","x2"])
node = construct_tree(x,y)
print_tree(node,feature_name)

Level 0
count of 0 = 1
count of 1 = 3
Current Entropy is = 0.8112781244591328
Splitting on feature x1 with gain ratio 0.31127812445913283 

Level 1
count of 0 = 1
count of 1 = 1
Current Entropy is = 1.0
Splitting on feature x2 with gain ratio 1.0 

Level 2
Count of 0 = 1
Current Entropy is = 0.0
Reached leaf Node 

Level 2
Count of 1 = 1
Current Entropy is = 0.0
Reached leaf Node 

Level 1
Count of 1 = 2
Current Entropy is = 0.0
Reached leaf Node 



# Trying above implemented decision tree on iris data

In [41]:
node = construct_tree(x_train,y_train)
print_tree(node,iris.feature_names)

Level 0
count of 0 = 37
count of 1 = 34
count of 2 = 41
Current Entropy is = 1.5807197138422104
Splitting on feature petal length (cm) with gain ratio 1.0 

Level 1
Count of 0 = 37
Current Entropy is = 0.0
Reached leaf Node 

Level 1
count of 1 = 34
count of 2 = 41
Current Entropy is = 0.993707106604508
Splitting on feature petal width (cm) with gain ratio 0.6610420198933152 

Level 2
count of 1 = 33
count of 2 = 4
Current Entropy is = 0.4941829348497886
Splitting on feature petal length (cm) with gain ratio 0.6941833044972409 

Level 3
Count of 1 = 32
Current Entropy is = 0.0
Reached leaf Node 

Level 3
count of 1 = 1
count of 2 = 4
Current Entropy is = 0.7219280948873623
Splitting on feature sepal length (cm) with gain ratio 0.3315597072868287 

Level 4
count of 1 = 1
count of 2 = 1
Current Entropy is = 1.0
Splitting on feature sepal width (cm) with gain ratio 1.0 

Level 5
Count of 2 = 1
Current Entropy is = 0.0
Reached leaf Node 

Level 5
Count of 1 = 1
Current Entropy is = 0.0
Rea

# Predicting the output of iris data using above implemented decision tree


In [42]:
predict = predict(x_test,node)
accuracy_score(predict,y_test)

0.9736842105263158

In [43]:
from sklearn.metrics import confusion_matrix

In [44]:
confusion_matrix(y_pred=predict, y_true=y_test)

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]], dtype=int64)

In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_pred=predict, y_true=y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

   micro avg       0.97      0.97      0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38



#  Decision tree using sklearn 

In [46]:
from sklearn.tree import DecisionTreeClassifier

In [47]:
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [48]:
predict = clf.predict(x_test)
accuracy_score(predict,y_test)

0.9736842105263158

In [49]:
confusion_matrix(y_pred=predict, y_true=y_test)

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]], dtype=int64)

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_pred=predict, y_true=y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

   micro avg       0.97      0.97      0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

